In [ ]:
import pandas as pd

df = pd.read_csv("C:/Users/Saher Raja/Documents/python-bank/data/transactions.csv")
df.head()


C:\Users\Saher Raja\AppData\Local\Temp\ipykernel_27800\2168566633.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:/Users/Saher Raja/Documents/python-bank/data/transactions.csv")


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes
0,f2f3b0fc-b7d7-4d85-b682-cbf07ef77c1a,2025-03-06 12:04:00,7746.03,SEK,SE8902EWFT80524424320740,SE8902ZUIH32054423564254,Sweden,Karlskrona,Sweden,Eskilstuna,outgoing,Online purchase
1,feccbd1e-7752-4452-a44a-901510252561,2025-03-31 01:37:00,45193.04,SEK,SE8902GTSI58472323675409,SE8902ERNA19462716907689,Sweden,Piteå,Sweden,Norrköping,outgoing,Tax refund
2,bc6a480c-e2e1-434d-830c-1cda3f913038,2025-04-29 22:58:00,33029.71,SEK,SE8902SEZT63377163386089,SE8902LOLQ30361773270969,Sweden,Östersund,Sweden,Karlskoga,incoming,Tax refund
3,b832aa2c-77f3-4c4f-9697-9776b8d74abb,2025-02-27 14:52:00,35994 .61,SEK,SE8902MGEE43594824630478,SE8902XIVG18316797897983,Sweden,Karlstad,Sweden,Helsingborg,incoming,NaN
4,0d32c6ef-508e-4f5d-bc64-341a0a9f498d,2025-01-03 22:29:00,32229.73,SEK,SE8902SZWO66860675966705,SE8902SUUS97732018069804,Sweden,Skellefteå,Sweden,Falun,outgoing,Tax refund


In [7]:
# Lägger till en kolumn som markerar om varje rad är giltig
df['is_valid'] = True

In [ ]:
# Konverterar kolumnen till numerisk (float). Ogiltiga värden blir NaN.
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')

In [ ]:
# Skapar en valideringskolumn om du inte gjort det
df['is_valid'] = True

# Flaggar negativa eller noll belopp
df.loc[df['amount'] <= 0, 'is_valid'] = False


In [11]:
# Kollar saknade valutor
df.loc[df['currency'].isna(), 'is_valid'] = False


In [12]:
# Kollar saknade datum
df.loc[df['timestamp'].isna(), 'is_valid'] = False

In [13]:
# Kollar om sender eller receiver konton saknas
df.loc[df['sender_account'].isna(), 'is_valid'] = False
df.loc[df['receiver_account'].isna(), 'is_valid'] = False

In [14]:
# Visar rader som är ogiltiga
df[df['is_valid'] == False]

,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,is_valid


In [ ]:
# Kollar transaktioner över ett tröskelvärde (t.ex. 1 miljon SEK)
df['suspicious_amount'] = df['amount'] > 1_000_000
df[df['suspicious_amount']]

,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,is_valid,suspicious_amount
1611,c2565519-df33-426e-a061-f18d3c3e96a1,2025-05-05 22:49:00,2217915.0,SEK,SE8902AAZZ32784603345278,SE8902GTLW57183159704787,Sweden,Gävle,Sweden,Örebro,outgoing,Gift to family member,True,True
1614,d4b33191-0de5-485b-b438-ef17305024fd,2025-03-24 10:14:00,3700879.0,SEK,SE8902VQJB24905491378975,SE8902BTEZ26557538989061,Sweden,Norrköping,Sweden,Kalmar,outgoing,Payment for invoice #6501,True,True


In [16]:
# Dubbletter baserat på transaction_id
df['is_duplicate'] = df.duplicated(subset='transaction_id', keep=False)
df[df['is_duplicate']]

,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,is_valid,suspicious_amount,is_duplicate


In [17]:
# Kollar sender och receiver IBAN-format
df['invalid_sender_iban'] = df['sender_account'].str.len() != 24
df['invalid_receiver_iban'] = df['receiver_account'].str.len() != 24

# Visar alla rader med felaktiga IBAN
df[df['invalid_sender_iban'] | df['invalid_receiver_iban']]


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,is_valid,suspicious_amount,is_duplicate,invalid_sender_iban,invalid_receiver_iban
10000,d3b4356c-140d-4a88-b373-faa955e668cc,2025-04-14 00:43:00,14390.24,SEK,GB37AGFD92854504240999,SE8902QHUH19874587550707,Sweden,Mölndal,Sweden,Kalmar,incoming,Payment for invoice #9922,True,False,False,True,False
10001,8b0538dc-8f51-4f68-bd79-c8a3230d4c9c,2025-04-14 01:38:00,24117.70,SEK,SE8902AAZZ32784603345278,GB27XJMX69969377460150,Sweden,Växjö,French Guiana,Morenofurt,incoming,Insurance payout,True,False,False,False,True
10002,b167fec6-2061-4c78-a5fd-0661ff977dc0,2025-02-16 09:56:00,2421.91,SEK,SE8902NQPR37480978954996,GB26BEUC92999342765512,Sweden,Skövde,Sweden,Alingsås,incoming,Tax refund,True,False,False,False,True
10003,126e3b30-30cb-4032-82ef-9865e89bd74f,2025-01-21 18:04:00,49351.30,SEK,GB47ZZZF26033259677500,SE8902FJRZ14744000547809,Sweden,Sundsvall,Sweden,Varberg,incoming,Payment for invoice #3846,True,False,False,True,False
10004,043cafad-7802-49db-b4e6-692042d803bb,2025-01-25 08:29:00,8744.97,SEK,SE8902XIMR45726412027481,GB44GTUF00341976370730,Sweden,Alingsås,Sweden,Östersund,outgoing,Freelance project payment,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,ea7e0cb2-fadb-496c-962b-7e54a5cfa1bc,2025-02-03 17:00:00,146.04,USD,GB39XHRG15445191862019,SE8902AQYT55466061009593,Mauritania,Jacksonville,Sweden,Nyköping,outgoing,Gift to family member,True,False,False,True,False
99996,4271c3c0-d3be-49f5-8154-ee60e2a1e582,2025-01-07 14:00:00,56.98,USD,GB97OHYP64183530899251,SE8902AQYT55466061009593,Guinea,Waltersport,Sweden,Skellefteå,incoming,NaN,True,False,False,True,False
99997,23574238-9739-4af4-a823-5b699812db4d,2025-05-11 13:00:00,128.34,USD,GB68PWPH34390331224741,SE8902AQYT55466061009593,Tokelau,Davidfurt,Sweden,Luleå,incoming,Gift to family member,True,False,False,True,False
99998,38e785ff-e460-419e-af07-595f5c67aaf6,2025-02-02 13:00:00,149.13,USD,GB30NDUZ96025046626638,SE8902AQYT55466061009593,Bulgaria,West Andrewside,Sweden,Skellefteå,outgoing,Loan repayment,True,False,False,True,False


In [19]:
print(df.columns)


Index(['transaction_id', 'timestamp', 'amount', 'currency', 'sender_account',
       'receiver_account', 'sender_country', 'sender_municipality',
       'receiver_country', 'receiver_municipality', 'transaction_type',
       'notes', 'is_valid', 'suspicious_amount', 'is_duplicate',
       'invalid_sender_iban', 'invalid_receiver_iban'],
      dtype='object')


In [20]:
from datetime import datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['is_future'] = df['timestamp'] > datetime.now()

In [21]:
df[['is_valid', 'suspicious_amount', 'is_duplicate', 'invalid_sender_iban', 'invalid_receiver_iban', 'is_future']].sum()


is_valid                 100000
suspicious_amount             2
is_duplicate                  0
invalid_sender_iban       42476
invalid_receiver_iban     47524
is_future                   809
dtype: int64

Valideringsresultat
is_valid: 100 000
Alla rader är initialt satta till giltiga 

suspicious_amount: 2
Endast två transaktioner över 1 000 000 

is_duplicate: 0
Inga dubbletter av transaction_id 

invalid_sender_iban: 42 476
Cirka 42 % av sändarkonton har ogiltig längd (bör vara exakt 24 tecken).

invalid_receiver_iban: 47 524
Nästan hälften av mottagarkonton har ogiltig längd.

is_future: 809
809 transaktioner har datum i framtiden, det behöver kontrolleras om det är rimligt eller felaktigt.



In [22]:
#Visar misstänkta transaktioner:
df[df['suspicious_amount']]


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,is_valid,suspicious_amount,is_duplicate,invalid_sender_iban,invalid_receiver_iban,is_future
1611,c2565519-df33-426e-a061-f18d3c3e96a1,2025-05-05 22:49:00,2217915.0,SEK,SE8902AAZZ32784603345278,SE8902GTLW57183159704787,Sweden,Gävle,Sweden,Örebro,outgoing,Gift to family member,True,True,False,False,False,False
1614,d4b33191-0de5-485b-b438-ef17305024fd,2025-03-24 10:14:00,3700879.0,SEK,SE8902VQJB24905491378975,SE8902BTEZ26557538989061,Sweden,Norrköping,Sweden,Kalmar,outgoing,Payment for invoice #6501,True,True,False,False,False,False


In [23]:
#Visar framtida transaktioner:
df[df['is_future']]


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,is_valid,suspicious_amount,is_duplicate,invalid_sender_iban,invalid_receiver_iban,is_future
301,1b5a870b-db8b-46f6-ab2d-b452fbb1a04e,2026-03-05 07:20:00,4669.37,SEK,SE8902ZZJJ01114055756075,SE8902SWUO93445136998779,Sweden,Sandviken,Sweden,Borlänge,incoming,Freelance project payment,True,False,False,False,False,True
355,26351bef-2cb1-46b6-891b-c6c98b41901a,2025-11-16 11:26:00,9221.77,SEK,SE8902IGJG42825638310983,SE8902QHUH19874587550707,Sweden,Karlstad,Sweden,Sundsvall,incoming,Utility bill,True,False,False,False,False,True
394,45af305a-0822-4e92-ab47-842abd9b4098,2025-07-06 23:47:00,17812.33,SEK,SE8902UDED11242220219330,SE8902IOSH72112913547339,Sweden,Karlskrona,Sweden,Ängelholm,outgoing,NaN,True,False,False,False,False,True
685,e3eb5932-595d-426b-9a6d-9235718b0d7c,2025-07-15 04:44:00,19996.02,SEK,SE8902WRYQ08007617557527,SE8902SPJM60038922234992,Sweden,Varberg,Sweden,Göteborg,outgoing,Freelance project payment,True,False,False,False,False,True
865,eed6c6a8-cbc3-411b-a768-5cb7a80f7581,2025-12-18 08:29:00,39746.22,SEK,SE8902ZUIH32054423564254,SE8902WLVP30891623838570,Sweden,Uppsala,Sweden,Eskilstuna,outgoing,Loan repayment,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99210,147b0a72-4f56-4b3e-a8fb-de02e4e35b5c,2025-08-10 17:02:15,167.83,SEK,SE8902AZCJ34918120321272,GB52OTJL95378550133263,Sweden,Göteborg,Germany,Benjaminton,incoming,Loan repayment,True,False,False,False,True,True
99307,d07c4b97-0666-4679-93a8-3deabcad9367,2026-02-19 17:01:57,293.36,SEK,SE8902SPDW36248491049541,GB70SKJH31024082026605,Sweden,Kristianstad,Germany,Coxfort,outgoing,Loan repayment,True,False,False,False,True,True
99472,7735a90f-7e01-46e6-8db1-e17171d873b9,2025-06-09 13:01:37,544.29,SEK,SE8902YHAA89862840296588,GB04TVEC07866665486898,Sweden,Karlstad,Germany,West Edwardmouth,incoming,Tax refund,True,False,False,False,True,True
99657,35b2d869-ede2-4672-ba9e-6212f497d04f,2025-10-08 17:01:25,790.58,SEK,SE8902VCCV99038245497441,GB47VJAO65682377592268,Sweden,Skövde,Germany,باغات آریا,incoming,Insurance payout,True,False,False,False,True,True


In [24]:
#Visar de med ogiltigt IBAN:
df[df['invalid_sender_iban'] | df['invalid_receiver_iban']]


,transaction_id,timestamp,amount,currency,sender_account,receiver_account,sender_country,sender_municipality,receiver_country,receiver_municipality,transaction_type,notes,is_valid,suspicious_amount,is_duplicate,invalid_sender_iban,invalid_receiver_iban,is_future
10000,d3b4356c-140d-4a88-b373-faa955e668cc,2025-04-14 00:43:00,14390.24,SEK,GB37AGFD92854504240999,SE8902QHUH19874587550707,Sweden,Mölndal,Sweden,Kalmar,incoming,Payment for invoice #9922,True,False,False,True,False,False
10001,8b0538dc-8f51-4f68-bd79-c8a3230d4c9c,2025-04-14 01:38:00,24117.70,SEK,SE8902AAZZ32784603345278,GB27XJMX69969377460150,Sweden,Växjö,French Guiana,Morenofurt,incoming,Insurance payout,True,False,False,False,True,False
10002,b167fec6-2061-4c78-a5fd-0661ff977dc0,2025-02-16 09:56:00,2421.91,SEK,SE8902NQPR37480978954996,GB26BEUC92999342765512,Sweden,Skövde,Sweden,Alingsås,incoming,Tax refund,True,False,False,False,True,False
10003,126e3b30-30cb-4032-82ef-9865e89bd74f,2025-01-21 18:04:00,49351.30,SEK,GB47ZZZF26033259677500,SE8902FJRZ14744000547809,Sweden,Sundsvall,Sweden,Varberg,incoming,Payment for invoice #3846,True,False,False,True,False,False
10004,043cafad-7802-49db-b4e6-692042d803bb,2025-01-25 08:29:00,8744.97,SEK,SE8902XIMR45726412027481,GB44GTUF00341976370730,Sweden,Alingsås,Sweden,Östersund,outgoing,Freelance project payment,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,ea7e0cb2-fadb-496c-962b-7e54a5cfa1bc,2025-02-03 17:00:00,146.04,USD,GB39XHRG15445191862019,SE8902AQYT55466061009593,Mauritania,Jacksonville,Sweden,Nyköping,outgoing,Gift to family member,True,False,False,True,False,False
99996,4271c3c0-d3be-49f5-8154-ee60e2a1e582,2025-01-07 14:00:00,56.98,USD,GB97OHYP64183530899251,SE8902AQYT55466061009593,Guinea,Waltersport,Sweden,Skellefteå,incoming,NaN,True,False,False,True,False,False
99997,23574238-9739-4af4-a823-5b699812db4d,2025-05-11 13:00:00,128.34,USD,GB68PWPH34390331224741,SE8902AQYT55466061009593,Tokelau,Davidfurt,Sweden,Luleå,incoming,Gift to family member,True,False,False,True,False,False
99998,38e785ff-e460-419e-af07-595f5c67aaf6,2025-02-02 13:00:00,149.13,USD,GB30NDUZ96025046626638,SE8902AQYT55466061009593,Bulgaria,West Andrewside,Sweden,Skellefteå,outgoing,Loan repayment,True,False,False,True,False,False


In [26]:
import os

# Skapar mappen om den inte finns
os.makedirs("output", exist_ok=True)


In [27]:
df[df['suspicious_amount']].to_csv("output/suspicious_amounts.csv", index=False)


In [28]:
# Sparar framtida transaktioner
df[df['is_future']].to_csv("output/future_transactions.csv", index=False)

In [29]:
# Sparar rader med felaktiga IBAN
df[df['invalid_sender_iban'] | df['invalid_receiver_iban']].to_csv("output/invalid_ibans.csv", index=False)